# Modelos de Regresion Espacial SAR

## Autocorrelación Endógena con la variable dependiente

In [6]:
import geopandas as gpd
import pandas as pd
import numpy as np
import libpysal
from libpysal.weights import Queen
from sklearn.preprocessing import StandardScaler
import spreg

In [2]:
gdf = gpd.read_file("https://github.com/algarciach/AnalisisGeoespacial/raw/main/Covid19_model/Data/covid19_municipios_antioquia.gpkg")
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   codigo_municipio     125 non-null    object  
 1   nombre_municipio     125 non-null    object  
 2   codigo_subregion     125 non-null    object  
 3   nombre_subregion     125 non-null    object  
 4   area_municipio       125 non-null    float64 
 5   altitud              125 non-null    float64 
 6   temperatura          125 non-null    float64 
 7   humedad_relativa     125 non-null    float64 
 8   poblacion            125 non-null    int64   
 9   densidad             125 non-null    float64 
 10  muertes_covid19      125 non-null    int64   
 11  recuperados_covid19  125 non-null    int64   
 12  cfr                  125 non-null    float64 
 13  geometry             125 non-null    geometry
dtypes: float64(6), geometry(1), int64(3), object(4)
memory usage: 13.8

In [7]:
gdf['density_cfr'] = (gdf['cfr'] / gdf['area_municipio'])
gdf['y'] = np.log(gdf['density_cfr'] + 1)

y = gdf['y'].values.reshape(-1, 1)
independent_vars = ['densidad', 'altitud', 'temperatura', 'humedad_relativa']
X = gdf[independent_vars].values

st = StandardScaler()
X_scaled = st.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=independent_vars)
w = Queen.from_dataframe(gdf)

wx_dict = {}
for var in independent_vars:
    wx_dict[f'w_{var}'] = libpysal.weights.spatial_lag.lag_spatial(w, X_scaled_df[var])

wx_df = pd.DataFrame(wx_dict)
slx_exog = pd.concat([X_scaled_df, wx_df], axis=1)

ols_model = spreg.OLS(gdf['y'].values.reshape((-1, 1)), 
                      slx_exog.values, 
                      name_y='log_density_cfr', 
                      name_x=slx_exog.columns.tolist(),
                      name_w='queen_contiguity')

print(ols_model.summary)

/var/folders/nq/kj34bm5140bgwbktdx1z94cm0000gn/T/ipykernel_25034/2596927528.py:11: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gdf)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :log_density_cfr                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           9
S.D. dependent var  :      0.0161                Degrees of Freedom    :         116
R-squared           :      0.2656
Adjusted R-squared  :      0.2150
Sum squared residual:   0.0237285                F-statistic           :      5.2449
Sigma-square        :       0.000                Prob(F-statistic)     :   1.301e-05
S.E. of regression  :       0.014                Log likelihood        :     358.220
Sigma-square ML     :       0.000                Akaike info criterion :    -698.439
S.E of regression ML:      0.0138                Schwarz criterion     :    -672.984

---------------------------------------------------------

```text
  w = Queen.from_dataframe(gdf)
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :log_density_cfr                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           9
S.D. dependent var  :      0.0161                Degrees of Freedom    :         116
R-squared           :      0.2656
Adjusted R-squared  :      0.2150
Sum squared residual:   0.0237285                F-statistic           :      5.2449
Sigma-square        :       0.000                Prob(F-statistic)     :   1.301e-05
S.E. of regression  :       0.014                Log likelihood        :     358.220
Sigma-square ML     :       0.000                Akaike info criterion :    -698.439
S.E of regression ML:      0.0138                Schwarz criterion     :    -672.984

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     t-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01507         0.00139        10.82174         0.00000
            densidad         0.00433         0.00171         2.53201         0.01268
             altitud         0.03940         0.01706         2.30955         0.02268
         temperatura         0.03610         0.01604         2.25107         0.02627
...
TEST                             DF        VALUE           PROB
Breusch-Pagan test                8         36.288           0.0000
Koenker-Bassett test              8          5.243           0.7313
================================ END OF REPORT =====================================
```
**Conclusiones**

Este modelo OLS es el primero que muestra la densidad poblacional como un predictor estadísticamente significativo, junto con la altitud y la temperatura. La relación con la variable dependiente transformada (log_density_cfr) es positiva para estas tres variables, sugiriendo que a medida que aumentan la densidad, altitud y temperatura, también lo hace la densidad del logaritmo de la Tasa de Letalidad.

Aunque el poder explicativo general (R-cuadrado del 26.56%) es moderado y el modelo es globalmente significativo, la indicación de heterocedasticidad por la prueba de Breusch-Pagan es una preocupación importante. La heterocedasticidad significa que los errores estándar y, por lo tanto, los p-valores y los intervalos de confianza, podrían ser incorrectos. Esto podría llevar a conclusiones engañosas sobre la significancia de tus variables.

Para abordar la heterocedasticidad, podrías considerar:

Utilizar errores estándar robustos.

Transformaciones adicionales de tus variables.

Modelos alternativos que sean más robustos a la heterocedasticidad (por ejemplo, ciertos tipos de GLM o modelos que modelen la varianza).

In [8]:
spatial_error_model = spreg.GM_Error_Het(y, X_scaled, w=w,
                                         name_y='log_density_cfr_rec',
                                         name_x=independent_vars,
                                         name_w='queen_contiguity')
print(spatial_error_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           5
S.D. dependent var  :      0.0161                Degrees of Freedom    :         120
Pseudo R-squared    :      0.2378
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01616         0.00200         8.07299         0.00000
            densidad         0.00532         

In [9]:
spatial_lag_model = spreg.GM_Lag(y, X_scaled, w=w,
                                 name_y='log_density_cfr_rec',
                                 name_x=independent_vars,
                                 name_w='queen_contiguity')
print(spatial_lag_model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :queen_contiguity
Dependent Variable  :log_density_cfr_rec                Number of Observations:         125
Mean dependent var  :      0.0146                Number of Variables   :           6
S.D. dependent var  :      0.0161                Degrees of Freedom    :         119
Pseudo R-squared    :      0.2334
Spatial Pseudo R-squared:  0.2270

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.01375         0.00428         3.21669         0.00130
            densidad         0.00565         0.00158         3.57898         0.00034
             altitud         0.